# Monte Carlo Uncertainty Analysis

This notebook demonstrates comprehensive Monte Carlo uncertainty quantification for hypersonic reentry trajectories.

## 🎯 Learning Objectives

- Understand uncertainty sources in hypersonic reentry
- Define uncertain parameters with statistical distributions
- Run Monte Carlo simulations with Latin Hypercube Sampling
- Analyze uncertainty propagation and statistical results
- Create uncertainty visualizations and confidence intervals

## 🔬 Uncertainty Sources

Key uncertainty sources in hypersonic reentry include:
- **Vehicle parameters**: Mass, drag/lift coefficients, geometry
- **Atmospheric conditions**: Density variations, temperature, winds
- **Initial conditions**: State estimation errors
- **Model uncertainty**: Aerodynamic and thermodynamic models

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add framework to path
sys.path.insert(0, str(Path('..') / 'src'))

# Import framework components
from hypersonic_reentry.dynamics import VehicleDynamics, VehicleState
from hypersonic_reentry.atmosphere import USStandard1976
from hypersonic_reentry.uncertainty import UncertaintyQuantifier, UncertainParameter
from hypersonic_reentry.utils.constants import DEG_TO_RAD, RAD_TO_DEG

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib available: {plt.__version__}")

## Step 1: Define Baseline Configuration

Establish the nominal vehicle and atmospheric configuration.

In [ ]:
# Define baseline vehicle parameters
vehicle_params = {
    'mass': 5000.0,              # kg
    'reference_area': 15.0,      # m²
    'drag_coefficient': 1.2,     # dimensionless
    'lift_coefficient': 0.8,     # dimensionless
    'ballistic_coefficient': 400.0,  # kg/m²
    'nose_radius': 0.5,          # m
    'length': 10.0,              # m
    'diameter': 2.0              # m
}

# Create atmosphere and dynamics
atmosphere = USStandard1976()
dynamics = VehicleDynamics(vehicle_params, atmosphere_model=atmosphere)

# Define baseline initial conditions
initial_state = VehicleState(
    altitude=120000.0,                    # 120 km
    latitude=28.5 * DEG_TO_RAD,          # KSC latitude
    longitude=-80.6 * DEG_TO_RAD,        # KSC longitude
    velocity=7800.0,                     # m/s
    flight_path_angle=-2.0 * DEG_TO_RAD, # degrees
    azimuth=90.0 * DEG_TO_RAD,          # eastward
    time=0.0
)

print("Baseline Configuration:")
print(f"Vehicle mass: {vehicle_params['mass']} kg")
print(f"Entry altitude: {initial_state.altitude/1000:.1f} km")
print(f"Entry velocity: {initial_state.velocity/1000:.1f} km/s")
print(f"Entry angle: {initial_state.flight_path_angle*RAD_TO_DEG:.1f}°")

## Step 2: Define Uncertain Parameters

Specify uncertain parameters with their probability distributions based on engineering judgment and available data.

In [ ]:
# Define uncertain parameters with realistic uncertainty levels
uncertain_params = [
    # Vehicle mass uncertainty (±5% - fuel consumption, payload variation)
    UncertainParameter(
        name="mass",
        distribution_type="normal",
        parameters={"mean": 5000.0, "std": 250.0},
        description="Vehicle mass with fuel/payload uncertainty"
    ),
    
    # Drag coefficient uncertainty (±10% - aerodynamic model uncertainty)
    UncertainParameter(
        name="drag_coefficient",
        distribution_type="normal",
        parameters={"mean": 1.2, "std": 0.12},
        description="Drag coefficient with aerodynamic uncertainty"
    ),
    
    # Lift coefficient uncertainty (±10%)
    UncertainParameter(
        name="lift_coefficient",
        distribution_type="normal", 
        parameters={"mean": 0.8, "std": 0.08},
        description="Lift coefficient with aerodynamic uncertainty"
    ),
    
    # Reference area uncertainty (±5% - manufacturing tolerance)
    UncertainParameter(
        name="reference_area",
        distribution_type="normal",
        parameters={"mean": 15.0, "std": 0.75},
        description="Reference area with manufacturing uncertainty"
    ),
    
    # Atmospheric density factor (±15% - atmospheric variability)
    UncertainParameter(
        name="atmospheric_density_factor",
        distribution_type="log_normal",
        parameters={"mean": 1.0, "std": 0.15},
        description="Atmospheric density scaling factor"
    )
]

print("Uncertain Parameters:")
print("=" * 60)
for param in uncertain_params:
    print(f"{param.name:25}: {param.distribution_type:12} - {param.description}")
    if param.distribution_type == "normal":
        mean = param.parameters["mean"]
        std = param.parameters["std"]
        cv = (std / mean) * 100
        print(f"{'':27} μ={mean:.2f}, σ={std:.3f} (CV={cv:.1f}%)")
    elif param.distribution_type == "log_normal":
        mean = param.parameters["mean"]
        std = param.parameters["std"]
        print(f"{'':27} μ={mean:.2f}, σ={std:.3f}")
    print()

## Step 3: Visualize Parameter Distributions

Plot the probability density functions of the uncertain parameters.

In [ ]:
# Create parameter distribution plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, param in enumerate(uncertain_params):
    ax = axes[i]
    
    if param.distribution_type == "normal":
        mean = param.parameters["mean"]
        std = param.parameters["std"]
        x = np.linspace(mean - 4*std, mean + 4*std, 1000)
        pdf = stats.norm.pdf(x, mean, std)
        
    elif param.distribution_type == "log_normal":
        mean = param.parameters["mean"]
        std = param.parameters["std"]
        # Convert to lognormal parameters
        mu = np.log(mean / np.sqrt(1 + (std/mean)**2))
        sigma = np.sqrt(np.log(1 + (std/mean)**2))
        x = np.linspace(0.01, mean + 4*std, 1000)
        pdf = stats.lognorm.pdf(x, sigma, scale=np.exp(mu))
    
    ax.plot(x, pdf, linewidth=2, label=f'{param.distribution_type.replace("_", " ").title()}')
    ax.fill_between(x, pdf, alpha=0.3)
    ax.set_xlabel(param.name.replace('_', ' ').title())
    ax.set_ylabel('Probability Density')
    ax.set_title(f'{param.name.replace("_", " ").title()} Distribution')
    ax.grid(True, alpha=0.3)
    ax.legend()

# Remove unused subplot
if len(uncertain_params) < len(axes):
    axes[-1].remove()

plt.suptitle('Uncertain Parameter Distributions', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("📊 Parameter distributions visualized")

## Step 4: Set Up Monte Carlo Analysis

Configure the uncertainty quantifier with advanced sampling techniques.

In [ ]:
# Create uncertainty quantifier
uq = UncertaintyQuantifier(
    vehicle_dynamics=dynamics,
    uncertain_parameters=uncertain_params,
    random_seed=42  # For reproducible results
)

print("🔬 Uncertainty Quantifier Configuration:")
print(f"Number of uncertain parameters: {len(uncertain_params)}")
print(f"Sampling method: Latin Hypercube Sampling (LHS)")
print(f"Random seed: 42 (reproducible results)")
print(f"Base vehicle dynamics: 3-DOF point mass")

# Display sampling information
print("\n📈 Sampling Strategy:")
print("- Latin Hypercube Sampling for better space-filling")
print("- Stratified sampling ensures coverage of parameter space")
print("- Each parameter divided into equal-probability intervals")
print("- Random permutation prevents parameter correlation")

## Step 5: Run Monte Carlo Simulation

Execute the Monte Carlo analysis with a moderate number of samples for demonstration.

In [ ]:
# Run Monte Carlo analysis
print("🚀 Starting Monte Carlo simulation...")
print("This may take several minutes depending on sample size and complexity.")

# Configure simulation parameters
num_samples = 500  # Moderate size for notebook demonstration
time_span = (0.0, 1800.0)  # 30 minutes simulation time
parallel = False  # Set to True if multiprocessing available

print(f"\nSimulation Parameters:")
print(f"Number of samples: {num_samples}")
print(f"Simulation time: {time_span[1]/60:.1f} minutes")
print(f"Parallel processing: {parallel}")

# Execute Monte Carlo analysis
mc_result = uq.run_monte_carlo_analysis(
    initial_state=initial_state,
    time_span=time_span,
    num_samples=num_samples,
    parallel=parallel
)

print(f"\n✅ Monte Carlo simulation completed!")
print(f"Successful simulations: {mc_result.num_samples}")
print(f"Available outputs: {list(mc_result.mean_values.keys())}")

# Display basic statistics
print("\n📊 Basic Statistics:")
for output_name, mean_val in mc_result.mean_values.items():
    std_val = mc_result.std_values[output_name]
    cv = (std_val / abs(mean_val)) * 100 if abs(mean_val) > 1e-10 else 0
    print(f"{output_name:20}: μ={mean_val:10.2e}, σ={std_val:10.2e}, CV={cv:6.1f}%")

## Step 6: Statistical Analysis of Results

Analyze the Monte Carlo results with statistical methods.

In [ ]:
# Extract key performance metrics for detailed analysis
outputs_of_interest = ['final_altitude', 'final_velocity', 'downrange', 'flight_time']
available_outputs = [output for output in outputs_of_interest if output in mc_result.mean_values]

print("📈 DETAILED STATISTICAL ANALYSIS")
print("=" * 50)

# Create DataFrame for easier analysis
if hasattr(mc_result, 'raw_data') and 'performance_metrics' in mc_result.raw_data:
    df = pd.DataFrame(mc_result.raw_data['performance_metrics'])
    
    for output in available_outputs:
        if output in df.columns:
            data = df[output].values
            
            print(f"\n{output.replace('_', ' ').title()}:")
            print(f"  Mean:       {np.mean(data):12.3e}")
            print(f"  Std Dev:    {np.std(data):12.3e}")
            print(f"  Minimum:    {np.min(data):12.3e}")
            print(f"  Maximum:    {np.max(data):12.3e}")
            print(f"  Median:     {np.median(data):12.3e}")
            print(f"  Skewness:   {stats.skew(data):12.3f}")
            print(f"  Kurtosis:   {stats.kurtosis(data):12.3f}")
            
            # Confidence intervals
            ci_95 = np.percentile(data, [2.5, 97.5])
            ci_90 = np.percentile(data, [5, 95])
            
            print(f"  95% CI:     [{ci_95[0]:10.3e}, {ci_95[1]:10.3e}]")
            print(f"  90% CI:     [{ci_90[0]:10.3e}, {ci_90[1]:10.3e}]")
            
else:
    print("Raw data not available for detailed analysis")
    print("Using summary statistics from Monte Carlo result")
    
    for output in available_outputs:
        if output in mc_result.mean_values:
            mean_val = mc_result.mean_values[output]
            std_val = mc_result.std_values[output]
            
            print(f"\n{output.replace('_', ' ').title()}:")
            print(f"  Mean:       {mean_val:12.3e}")
            print(f"  Std Dev:    {std_val:12.3e}")
            print(f"  CV:         {(std_val/abs(mean_val)*100):12.1f}%")
            
            # Approximate confidence intervals (assuming normal distribution)
            ci_95 = [mean_val - 1.96*std_val, mean_val + 1.96*std_val]
            ci_90 = [mean_val - 1.645*std_val, mean_val + 1.645*std_val]
            
            print(f"  95% CI:     [{ci_95[0]:10.3e}, {ci_95[1]:10.3e}]")
            print(f"  90% CI:     [{ci_90[0]:10.3e}, {ci_90[1]:10.3e}]")

## Step 7: Create Uncertainty Visualizations

Generate comprehensive plots to understand uncertainty propagation.

In [ ]:
# Create comprehensive uncertainty visualization
if hasattr(mc_result, 'raw_data') and 'performance_metrics' in mc_result.raw_data:
    df = pd.DataFrame(mc_result.raw_data['performance_metrics'])
    
    # Set up the plotting layout
    fig = plt.figure(figsize=(16, 12))
    
    # Define outputs to plot
    plot_outputs = [col for col in df.columns if col in available_outputs][:4]
    
    if len(plot_outputs) >= 2:
        # 1. Histogram plots
        for i, output in enumerate(plot_outputs[:4]):
            plt.subplot(3, 2, i+1)
            
            data = df[output].values
            
            # Plot histogram with KDE
            plt.hist(data, bins=30, alpha=0.7, density=True, color=f'C{i}', edgecolor='black')
            
            # Add KDE curve
            try:
                kde = stats.gaussian_kde(data)
                x_kde = np.linspace(np.min(data), np.max(data), 200)
                plt.plot(x_kde, kde(x_kde), 'r-', linewidth=2, label='KDE')
            except:
                pass
            
            # Add statistical information
            mean_val = np.mean(data)
            std_val = np.std(data)
            plt.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2e}')
            plt.axvline(mean_val-std_val, color='orange', linestyle=':', alpha=0.7)
            plt.axvline(mean_val+std_val, color='orange', linestyle=':', alpha=0.7, label='±1σ')
            
            plt.xlabel(output.replace('_', ' ').title())
            plt.ylabel('Probability Density')
            plt.title(f'{output.replace("_", " ").title()} Distribution')
            plt.legend(fontsize=8)
            plt.grid(True, alpha=0.3)
        
        # 5. Correlation matrix
        if len(plot_outputs) >= 2:
            plt.subplot(3, 2, 5)
            correlation_matrix = df[plot_outputs].corr()
            im = plt.imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
            plt.colorbar(im)
            plt.xticks(range(len(plot_outputs)), [col.replace('_', '\n') for col in plot_outputs], rotation=45)
            plt.yticks(range(len(plot_outputs)), [col.replace('_', '\n') for col in plot_outputs])
            plt.title('Output Correlation Matrix')
            
            # Add correlation values
            for i in range(len(plot_outputs)):
                for j in range(len(plot_outputs)):
                    plt.text(j, i, f'{correlation_matrix.iloc[i,j]:.2f}', 
                            ha='center', va='center', fontsize=8,
                            color='white' if abs(correlation_matrix.iloc[i,j]) > 0.5 else 'black')
        
        # 6. Scatter plot of two key outputs
        if len(plot_outputs) >= 2:
            plt.subplot(3, 2, 6)
            x_data = df[plot_outputs[0]].values
            y_data = df[plot_outputs[1]].values
            
            plt.scatter(x_data, y_data, alpha=0.6, s=20)
            plt.xlabel(plot_outputs[0].replace('_', ' ').title())
            plt.ylabel(plot_outputs[1].replace('_', ' ').title())
            plt.title(f'{plot_outputs[0]} vs {plot_outputs[1]}'.replace('_', ' ').title())
            plt.grid(True, alpha=0.3)
            
            # Add correlation coefficient
            corr_coeff = np.corrcoef(x_data, y_data)[0, 1]
            plt.text(0.05, 0.95, f'Correlation: {corr_coeff:.3f}', 
                    transform=plt.gca().transAxes, bbox=dict(boxstyle='round', facecolor='wheat'))
    
    plt.suptitle('Monte Carlo Uncertainty Analysis Results', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️ Detailed raw data not available for comprehensive visualization")
    print("Creating summary plots based on available statistics...")
    
    # Create simple bar plot of coefficients of variation
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot means
    outputs = list(mc_result.mean_values.keys())
    means = list(mc_result.mean_values.values())
    
    ax1.bar(range(len(outputs)), means)
    ax1.set_xticks(range(len(outputs)))
    ax1.set_xticklabels([o.replace('_', '\n') for o in outputs], rotation=45)
    ax1.set_ylabel('Mean Value')
    ax1.set_title('Mean Performance Metrics')
    ax1.grid(True, alpha=0.3)
    
    # Plot coefficients of variation
    cvs = [(mc_result.std_values[o]/abs(mc_result.mean_values[o]))*100 
           for o in outputs if abs(mc_result.mean_values[o]) > 1e-10]
    
    ax2.bar(range(len(cvs)), cvs, color='orange')
    ax2.set_xticks(range(len(cvs)))
    ax2.set_xticklabels([o.replace('_', '\n') for o in outputs if abs(mc_result.mean_values[o]) > 1e-10], rotation=45)
    ax2.set_ylabel('Coefficient of Variation (%)')
    ax2.set_title('Relative Uncertainty (CV)')
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle('Monte Carlo Analysis Summary', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("📊 Uncertainty visualizations created successfully!")

## Step 8: Sensitivity Analysis Preview

Perform a basic correlation-based sensitivity analysis to understand parameter importance.

In [ ]:
# Perform correlation-based sensitivity analysis
print("🔍 PARAMETER SENSITIVITY ANALYSIS")
print("=" * 45)

if hasattr(mc_result, 'raw_data') and 'input_parameters' in mc_result.raw_data:
    # Create combined DataFrame with inputs and outputs
    input_df = pd.DataFrame(mc_result.raw_data['input_parameters'])
    output_df = pd.DataFrame(mc_result.raw_data['performance_metrics'])
    
    print("\nCorrelation-based Parameter Importance:")
    print("(Correlation between input parameters and key outputs)")
    print()
    
    # Calculate correlations for each output
    for output in available_outputs[:3]:  # Limit to first 3 outputs
        if output in output_df.columns:
            print(f"\n{output.replace('_', ' ').title()}:")
            print("-" * 30)
            
            correlations = []
            for param in uncertain_params:
                param_name = param.name
                if param_name in input_df.columns:
                    corr = np.corrcoef(input_df[param_name], output_df[output])[0, 1]
                    correlations.append((param_name, abs(corr), corr))
            
            # Sort by absolute correlation
            correlations.sort(key=lambda x: x[1], reverse=True)
            
            for param_name, abs_corr, corr in correlations:
                importance = "High" if abs_corr > 0.5 else "Medium" if abs_corr > 0.3 else "Low"
                direction = "↑" if corr > 0 else "↓" if corr < 0 else "○"
                print(f"{param_name:25}: {corr:6.3f} ({abs_corr:5.3f}) {direction} [{importance}]")
    
    # Create sensitivity heatmap
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Prepare correlation matrix
    param_names = [p.name for p in uncertain_params if p.name in input_df.columns]
    output_names = [o for o in available_outputs if o in output_df.columns]
    
    corr_matrix = np.zeros((len(output_names), len(param_names)))
    
    for i, output in enumerate(output_names):
        for j, param in enumerate(param_names):
            corr_matrix[i, j] = np.corrcoef(input_df[param], output_df[output])[0, 1]
    
    # Plot heatmap
    im = ax.imshow(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1, aspect='auto')
    
    # Set ticks and labels
    ax.set_xticks(range(len(param_names)))
    ax.set_yticks(range(len(output_names)))
    ax.set_xticklabels([p.replace('_', '\n') for p in param_names], rotation=45)
    ax.set_yticklabels([o.replace('_', '\n') for o in output_names])
    
    # Add correlation values
    for i in range(len(output_names)):
        for j in range(len(param_names)):
            text = ax.text(j, i, f'{corr_matrix[i, j]:.2f}',
                          ha="center", va="center", color="black" if abs(corr_matrix[i, j]) < 0.5 else "white")
    
    ax.set_title('Parameter-Output Correlation Matrix')
    plt.colorbar(im, ax=ax, label='Correlation Coefficient')
    plt.tight_layout()
    plt.show()
    
else:
    print("\n⚠️ Input parameter data not available for sensitivity analysis")
    print("This typically requires saving intermediate results during Monte Carlo simulation")
    print("\nFor comprehensive sensitivity analysis, see notebook: 05_sensitivity_analysis.ipynb")

print("\n📈 Sensitivity analysis completed!")

## 🎯 Key Findings and Insights

Based on the Monte Carlo analysis, we can draw several important conclusions:

In [ ]:
# Summarize key findings
print("🔍 KEY FINDINGS FROM MONTE CARLO ANALYSIS")
print("=" * 50)

print("\n1. 📊 STATISTICAL PROPERTIES:")
for output in available_outputs[:3]:
    if output in mc_result.mean_values:
        mean_val = mc_result.mean_values[output]
        std_val = mc_result.std_values[output]
        cv = (std_val / abs(mean_val)) * 100 if abs(mean_val) > 1e-10 else 0
        
        print(f"   {output.replace('_', ' ').title()}:")
        print(f"     - Coefficient of Variation: {cv:.1f}%")
        
        uncertainty_level = "High" if cv > 20 else "Medium" if cv > 10 else "Low"
        print(f"     - Uncertainty Level: {uncertainty_level}")
        print()

print("2. 🎯 DESIGN IMPLICATIONS:")
print("   - Parameters with CV > 15% require careful design margins")
print("   - Strong correlations indicate coupled design dependencies")
print("   - Non-normal distributions suggest nonlinear system behavior")
print()

print("3. 🔧 RECOMMENDATIONS:")
print("   - Focus uncertainty reduction on most influential parameters")
print("   - Design robust control systems for high-uncertainty outputs")
print("   - Use confidence intervals for mission planning and safety margins")
print("   - Consider Monte Carlo results in optimization formulations")
print()

print("4. 📈 NEXT STEPS:")
print("   - Increase sample size for production analysis (1000+ samples)")
print("   - Perform Sobol sensitivity analysis for variance decomposition")
print("   - Include more uncertainty sources (atmospheric, model uncertainty)")
print("   - Validate results with experimental data when available")

print(f"\n\n🎉 Monte Carlo analysis completed with {mc_result.num_samples} samples!")
print(f"📊 Statistical confidence achieved for design decision-making")
print(f"🚀 Ready for robust trajectory optimization and mission planning")

## 🚀 Next Steps

Now that you've mastered Monte Carlo uncertainty analysis:

1. **Trajectory Optimization**: `04_trajectory_optimization.ipynb`
   - Incorporate uncertainty in optimization formulations
   - Robust optimization under uncertainty

2. **Advanced Sensitivity Analysis**: `05_sensitivity_analysis.ipynb`
   - Sobol indices for variance-based sensitivity
   - Morris screening for factor prioritization

3. **Custom Analysis**: Extend this notebook by:
   - Adding more uncertain parameters
   - Increasing sample size for production runs
   - Including atmospheric uncertainty models
   - Implementing advanced sampling techniques

## 📚 References

- Saltelli, A. et al. "Global Sensitivity Analysis: The Primer" (2008)
- McKay, M.D. et al. "A Comparison of Three Methods for Selecting Values of Input Variables" (1979)
- Sobol, I.M. "Global sensitivity indices for nonlinear mathematical models" (2001)

**Happy uncertainty quantification! 🎲📊**